# 1193. Monthly Transactions I

SQL SchemaPandas SchemaTable: Transactions+---------------+---------+| Column Name   | Type    |+---------------+---------+| id            | int     || country       | varchar || state         | enum    || amount        | int     || trans_date    | date    |+---------------+---------+id is the primary key of this table.The table has information about incoming transactions.The state column is an enum of type ["approved", "declined"]. Write an SQL query to find for each month and country, the number of transactions and their total amount, the number of approved transactions and their total amount.Return the result table in any order.The query result format is in the following example. **Example 1:**Input: Transactions table:+------+---------+----------+--------+------------+| id   | country | state    | amount | trans_date |+------+---------+----------+--------+------------+| 121  | US      | approved | 1000   | 2018-12-18 || 122  | US      | declined | 2000   | 2018-12-19 || 123  | US      | approved | 2000   | 2019-01-01 || 124  | DE      | approved | 2000   | 2019-01-07 |+------+---------+----------+--------+------------+Output: +----------+---------+-------------+----------------+--------------------+-----------------------+| month    | country | trans_count | approved_count | trans_total_amount | approved_total_amount |+----------+---------+-------------+----------------+--------------------+-----------------------+| 2018-12  | US      | 2           | 1              | 3000               | 1000                  || 2019-01  | US      | 1           | 1              | 2000               | 2000                  || 2019-01  | DE      | 1           | 1              | 2000               | 2000                  |+----------+---------+-------------+----------------+--------------------+-----------------------+

## Solution Explanation
This problem requires us to aggregate transaction data by month and country, calculating several metrics:1. Total number of transactions2. Number of approved transactions3. Total amount of all transactions4. Total amount of approved transactionsThe key to solving this problem is to:1. Extract the month from the transaction date using DATE_FORMAT or similar function2. Group the data by month and country3. Count transactions and sum amounts for both all transactions and approved transactionsFor the SQL solution, we'll use GROUP BY with month and country, and use conditional aggregation (SUM with CASE WHEN) to calculate the approved transaction metrics.For the Pandas solution, we'll:1. Extract the month from the trans_date column2. Use groupby to group by month and country3. Use aggregation functions to calculate the required metrics

In [ ]:
import pandas as pddef monthly_transactions(transactions: pd.DataFrame) -> pd.DataFrame:    # Extract month from trans_date    transactions['month'] = transactions['trans_date'].dt.strftime('%Y-%m')        # Group by month and country and calculate metrics    result = transactions.groupby(['month', 'country']).agg(        trans_count=('id', 'count'),        approved_count=('state', lambda x: (x == 'approved').sum()),        trans_total_amount=('amount', 'sum'),        approved_total_amount=('amount', lambda x: transactions.loc[transactions['state'] == 'approved', 'amount'].sum())    ).reset_index()        # Fix the approved_total_amount calculation    # The previous lambda function was incorrect as it didn't respect the groupby    for idx, row in result.iterrows():        month, country = row['month'], row['country']        approved_amount = transactions[(transactions['month'] == month) &                                       (transactions['country'] == country) &                                       (transactions['state'] == 'approved')]['amount'].sum()        result.at[idx, 'approved_total_amount'] = approved_amount        return resultWait, there's an issue with the approved_total_amount calculation. Let me correct it:import pandas as pddef monthly_transactions(transactions: pd.DataFrame) -> pd.DataFrame:    # Extract month from trans_date    transactions['month'] = transactions['trans_date'].dt.strftime('%Y-%m')        # Group by month and country and calculate metrics    result = transactions.groupby(['month', 'country']).agg(        trans_count=('id', 'count'),        trans_total_amount=('amount', 'sum')    ).reset_index()        # Calculate approved metrics    approved_df = transactions[transactions['state'] == 'approved']    approved_metrics = approved_df.groupby(['month', 'country']).agg(        approved_count=('id', 'count'),        approved_total_amount=('amount', 'sum')    ).reset_index()        # Merge the results    result = result.merge(approved_metrics, on=['month', 'country'], how='left')        # Fill NaN values with 0 for cases where there are no approved transactions    result['approved_count'] = result['approved_count'].fillna(0).astype(int)    result['approved_total_amount'] = result['approved_total_amount'].fillna(0).astype(int)        return result

## Time and Space Complexity
* *Time Complexity**: O(n), where n is the number of transactions. We need to iterate through all transactions once to extract the month and perform the groupby operations.* *Space Complexity**: O(m), where m is the number of unique month-country combinations. In the worst case, if all transactions are in different month-country combinations, m could be equal to n. We create a few intermediate dataframes:1. The original dataframe with an additional 'month' column: O(n)2. The result dataframe after groupby: O(m)3. The approved_df dataframe: O(n) in worst case4. The approved_metrics dataframe: O(m)So overall, the space complexity is O(n + m), which simplifies to O(n).

## Test Cases


In [ ]:
import pandas as pdfrom datetime import datetime# Test Case 1: Example from the problemdef test_example_case():    data = {        'id': [121, 122, 123, 124],        'country': ['US', 'US', 'US', 'DE'],        'state': ['approved', 'declined', 'approved', 'approved'],        'amount': [1000, 2000, 2000, 2000],        'trans_date': [datetime(2018, 12, 18), datetime(2018, 12, 19),                       datetime(2019, 1, 1), datetime(2019, 1, 7)]    }    transactions = pd.DataFrame(data)    result = monthly_transactions(transactions)        # Check the result has the correct shape    assert result.shape == (3, 6)        # Check specific values    us_dec = result[(result['month'] == '2018-12') & (result['country'] == 'US')].iloc[0]    assert us_dec['trans_count'] == 2    assert us_dec['approved_count'] == 1    assert us_dec['trans_total_amount'] == 3000    assert us_dec['approved_total_amount'] == 1000        print("Test case 1 passed!")# Test Case 2: Empty dataframedef test_empty_dataframe():    transactions = pd.DataFrame(columns=['id', 'country', 'state', 'amount', 'trans_date'])    result = monthly_transactions(transactions)    assert result.empty    print("Test case 2 passed!")# Test Case 3: All transactions approveddef test_all_approved():    data = {        'id': [1, 2, 3],        'country': ['US', 'US', 'DE'],        'state': ['approved', 'approved', 'approved'],        'amount': [100, 200, 300],        'trans_date': [datetime(2019, 1, 1), datetime(2019, 1, 15), datetime(2019, 2, 1)]    }    transactions = pd.DataFrame(data)    result = monthly_transactions(transactions)        # Check that approved counts match transaction counts    assert all(result['trans_count'] == result['approved_count'])    assert all(result['trans_total_amount'] == result['approved_total_amount'])    print("Test case 3 passed!")# Test Case 4: All transactions declineddef test_all_declined():    data = {        'id': [1, 2, 3],        'country': ['US', 'US', 'DE'],        'state': ['declined', 'declined', 'declined'],        'amount': [100, 200, 300],        'trans_date': [datetime(2019, 1, 1), datetime(2019, 1, 15), datetime(2019, 2, 1)]    }    transactions = pd.DataFrame(data)    result = monthly_transactions(transactions)        # Check that approved counts are all 0    assert all(result['approved_count'] == 0)    assert all(result['approved_total_amount'] == 0)    print("Test case 4 passed!")# Run the teststest_example_case()test_empty_dataframe()test_all_approved()test_all_declined()